# Risk and reliability model ( rrm )

In [1]:
# Import auxiliary libraries for demonstration

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

plt.rcParams[ "figure.figsize" ] = [ 5, 4 ]

plt.rcParams[ "figure.dpi" ] = 80
plt.rcParams[ "font.family" ] = "Times New Roman"
plt.rcParams[ "font.size" ] = '14'

## First order second moment

### FOSM with mean value algorithm

FOSM with mean value algorithm can be expressed by the following equation,

$$
\beta = \frac{g(\mu_1, \mu_2, \dots, \mu_n)}{\sqrt{ \sum_{i=1}^n \alpha_i \sigma_i}}
$$

where $g$ is the limit state function (LSF); $\alpha_i$ is given by,

$$
\alpha_i = \frac{\partial g}{ \partial X_i} \Bigr|_{\mu_i}
$$

Function `fosmMVAL` implements the FOSM with mean value algorithm.

Reference: Nowak, A.S. and Collins, K.R., 2012. Reliability of structures. CRC press.

#### Function help

In [2]:
from ffpack.rrm import fosmMVAL
help( fosmMVAL )

Help on function fosmMVAL in module ffpack.rrm.firstOrderSecondMoment:

fosmMVAL(dim, g, dg, mus, sigmas)
    First order second moment method based on mean value algorithm.
    
    Parameters
    ----------
    dim: integer
        Space dimension ( number of random variables ).
    g: function
        Limit state function. It will be called like g( [ x1, x2, ... ] ).
    dg: array_like of function 
        Gradient of the limit state function. It should be an array_like of function
        like dg = [ dg_dx1, dg_dx2, ... ]. To get the derivative of i-th random 
        variable at ( x1*, x2*, ... ), dg[ i ]( x1*, x2*, ... ) will be called.
        dg can be None, see the following Notes.
    mus: 1d array
        Mean of the random variables.
    sigmas: 1d array
        Variance of the random variables.
    
    Returns
    -------
    beta: scalar
        Reliability index.
    pf: scalar
        probability of failure.
    
    Raises
    ------
    ValueError
        If the dim 

#### Example with explicit derivative of LSF

In [3]:
# Define the dimension for the FOSM problem
dim = 2

# Define the limit state function (LSF) g
g = lambda X: 3 * X[ 0 ] - 2 * X[ 1 ]

# Explicit derivative of LSF
# dg is a list in which each element is a partial derivative function of g w.r.t. X
# dg[0] = partial g / partial X[0]
# dg[1] = partial g / partial X[1]
dg = [ lambda X: 3, lambda X: -2 ]

# Mean and standard deviation of the random variables
mus = [ 1, 1 ]
sigmas = [ 3, 4 ]

# Use mean value algorithm to get results
beta, pf = fosmMVAL( dim, g, dg, mus, sigmas)

In [4]:
print( "Reliability index: " )
print( beta )
print()
print( "Failure probability: " )
print( pf )

Reliability index: 
0.08304547985373997

Failure probability: 
0.46690768839408386


#### Example with automatic differentiation of LSF

In [5]:
# Define the dimension for the FOSM problem
dim = 2

# Define the limit state function (LSF) g
g = lambda X: 3 * X[ 0 ] - 2 * X[ 1 ]

# If dg is None, the internal automatic differentiation algorithm will be used
dg = None

# Mean and standard deviation of the random variables
mus = [ 1, 1 ]
sigmas = [ 3, 4 ]

# Use mean value algorithm to get results
beta, pf = fosmMVAL( dim, g, dg, mus, sigmas)

In [6]:
print( "Reliability index: " )
print( beta )
print()
print( "Failure probability: " )
print( pf )

Reliability index: 
0.08304547985853711

Failure probability: 
0.46690768839217667


## First order reliability method

### FORM with Hasofer-Lind-Rackwitz-Fiessler algorithm

Hasofer-Lind-Rackwitz-Fiessler (HLRF) algorithm is an iterative method to find the reliability index. It is shown that the HLRF algorithm is very effective in many situations even if the convergence is not assured in all cases. 

Function `formHLRF` implements the FORM with Hasofer-Lind-Rackwitz-Fiessler algorithm. The Nataf transformation is used in the method to map the random variables from X space to U space.

Reference:

1. Wang, C., 2021. Structural reliability and time-dependent reliability. Cham, Switzerland: Springer.
2. Lemaire, M., 2013. Structural reliability. John Wiley & Sons.

#### Function help

In [7]:
from ffpack.rrm import formHLRF
help( formHLRF )

Help on function formHLRF in module ffpack.rrm.firstOrderReliabilityMethod:

formHLRF(dim, g, dg, distObjs, corrMat, iter=1000, tol=1e-06, quadDeg=99, quadRange=8)
    First order reliability method based on Hasofer-Lind-Rackwitz-Fiessler algorithm.
    
    Parameters
    ----------
    dim: integer
        Space dimension ( number of random variables ).
    g: function
        Limit state function. It will be called like g( [ x1, x2, ... ] ).
    dg: array_like of function 
        Gradient of the limit state function. It should be an array_like of function
        like dg = [ dg_dx1, dg_dx2, ... ]. To get the derivative of i-th random 
        variable at ( x1*, x2*, ... ), dg[ i ]( x1*, x2*, ... ) will be called.
        dg can be None, see the following Notes.
    distObjs: array_like of distributions
        Marginal distribution objects. It should be the freezed distribution 
        objects with pdf, cdf, ppf. We recommend to use scipy.stats functions.
    corrMat: 2d matrix
  

#### Example with explicit derivative of LSF

In [8]:
# Define the dimension for the FORM problem
dim = 2

# Define the limit state function (LSF) g
g = lambda X: 1.0 - X[ 0 ] - X[ 1 ]

# Explicit derivative of LSF
# dg is a list in which each element is a partial derivative function of g w.r.t. X
# dg[0] = partial g / partial X[0]
# dg[1] = partial g / partial X[1]
dg = [ lambda X: -1, lambda X: -1 ]

# Marginal distributions and correlation Matrix of the random variables
distObjs = [ stats.norm(), stats.norm() ]
corrMat = np.eye( dim )

# Use Hasofer-Lind-Rackwitz-Fiessler algorithm to get results
beta, pf, uCoord, xCoord = formHLRF( dim, g, dg, distObjs, corrMat )

In [9]:
print( "Reliability index: " )
print( beta )
print()
print( "Failure probability: " )
print( pf )
print()
print( "Design point coordinate in U space: " )
print( uCoord )
print()
print( "Design point coordinate in X space: " )
print( xCoord )

Reliability index: 
0.7071067811865477

Failure probability: 
0.23975006109347669

Design point coordinate in U space: 
[0.5 0.5]

Design point coordinate in X space: 
[0.5 0.5]


#### Example with automatic differentiation of LSF

In [10]:
# Define the dimension for the FORM problem
dim = 2

# Define the limit state function (LSF) g
g = lambda X: 1.0 - X[ 0 ] - X[ 1 ]

# If dg is None, the internal automatic differentiation algorithm will be used
dg = None

# Marginal distributions and correlation Matrix of the random variables
distObjs = [ stats.norm(), stats.norm() ]
corrMat = np.eye( dim )

# Use Hasofer-Lind-Rackwitz-Fiessler algorithm to get results
beta, pf, uCoord, xCoord = formHLRF( dim, g, dg, distObjs, corrMat )

In [11]:
print( "Reliability index: " )
print( beta )
print()
print( "Failure probability: " )
print( pf )
print()
print( "Design point coordinate in U space: " )
print( uCoord )
print()
print( "Design point coordinate in X space: " )
print( xCoord )

Reliability index: 
0.7071053669729852

Failure probability: 
0.23975050048498586

Design point coordinate in U space: 
[0.499999 0.499999]

Design point coordinate in X space: 
[0.499999 0.499999]


### FORM with constrained optimization algorithm

Finding the design point or most probable point (MPP) for a given limit state function is basically a constrained optimization problem. It can be represented by the following equation,

$$
\beta = \underset{\beta}{\operatorname{argmin}}||\mathbf{U}||
$$

$$
s.t. g( \mathbf{X} ) = g( \mathbf{T}^{-1} ( \mathbf{U} ) ) = g( \mathbf{U} ) = 0
$$

where transformation $\mathbf{T}$ is introduced to map the original random variables $\mathbf{X}$ (in X-space) to the standard, uncorrelated normal variables $\mathbf{U}$ with $\mathbf{U} = \mathbf{T} ( \mathbf{X} )$.

Function `formCOPT` implements the FORM with constrained optimization algorithm. The Nataf transformation is used in the method to map the random variables from X space to U space.

#### Function help

In [12]:
from ffpack.rrm import formCOPT
help( formCOPT )

Help on function formCOPT in module ffpack.rrm.firstOrderReliabilityMethod:

formCOPT(dim, g, distObjs, corrMat, quadDeg=99, quadRange=8)
    First order reliability method based on constrained optimization.
    
    Parameters
    ----------
    dim: integer
        Space dimension ( number of random variables ).
    g: function
        Limit state function. It will be called like g( [ x1, x2, ... ] ).
    distObjs: array_like of distributions
        Marginal distribution objects. It should be the freezed distribution 
        objects with pdf, cdf, ppf. We recommend to use scipy.stats functions.
    corrMat: 2d matrix
        Correlation matrix of the marginal distributions.
    quadDeg: integer
        Quadrature degree for Nataf transformation
    quadRange: scalar
        Quadrature range for Nataf transformation. The integral will be performed 
        in the range [ -quadRange, quadRange ].
    
    Returns
    -------
    beta: scalar
        Reliability index.
    pf: scalar


#### Example with default values

In [13]:
# Define the dimension for the FORM problem
dim = 2

# Define the limit state function (LSF) g
g = lambda X: 1.0 - X[ 0 ] - X[ 1 ]

# Marginal distributions and correlation Matrix of the random variables
distObjs = [ stats.norm(), stats.norm() ]
corrMat = np.eye( dim )

# Use Hasofer-Lind-Rackwitz-Fiessler algorithm to get results
beta, pf, uCoord, xCoord = formCOPT( dim, g, distObjs, corrMat )

In [14]:
print( "Reliability index: " )
print( beta )
print()
print( "Failure probability: " )
print( pf )
print()
print( "Design point coordinate in U space: " )
print( uCoord )
print()
print( "Design point coordinate in X space: " )
print( xCoord )

Reliability index: 
0.7071067706498352

Failure probability: 
0.23975006436719704

Design point coordinate in U space: 
[0.49999999 0.49999999]

Design point coordinate in X space: 
[0.49999999 0.49999999]
